# Run the pipeline from the notebook


- work with Weakly_2022_39
- use jupyter kernel LSST

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/24


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_39/atmospec 	setup
drp_pipe              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_39/drp_pipe 	setup
eups                  LOCAL:/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/eups 	setup
rapid_analysis        LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_39/rapid_analysis 	setup


In [2]:
# Basic imports
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from pprint import pprint

In [3]:
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline

In [4]:
from lsst.daf.butler import Butler

repo = "/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/data/butler.yaml"

butler = Butler(repo)
registry = butler.registry

### DRP pipeline for Latiss
https://github.com/lsst/drp_pipe/tree/main/pipelines/LATISS

In [5]:
# set up an output collection with your username
#outputCollection = "u/nate2/analysisToolsExample"
inputCollections = ["LATISS/raw/all","refcats","LATISS/calib"]
outputCollection = "u/dagoret/test_latiss_drp"

# this can be skipped if you already have a read writable butler setup (above is read only)
butlerRW = SimplePipelineExecutor.prep_butler(repo, inputs = inputCollections,output=outputCollection)

# load in the pipeline to run
#pipeline = Pipeline.from_uri("$ANALYSIS_TOOLS_DIR/pipelines/coaddQualityCore.yaml")


pipeline = Pipeline.from_uri("$DRP_PIPE_DIR/pipelines/LATISS/DRP.yaml")

In [6]:
# override a configuration within a certain AnalysisTool
#configKey = "plots.shapeSizeFractionalDiffScatter.prep.selectors.snSelector.threshold"
#pipeline.addConfigOverride("analyzeObjectTableCore", configKey, 400)

#bands = ['g', 'r', 'i', 'z']
#pipeline.addConfigOverride("analyzeObjectTableCore", "bands", bands)
#pipeline.addConfigOverride("catalogMatchTract", "bands", bands)
#pipeline.addConfigOverride("refCatObjectTract", "bands", bands)

# restrict processing to the same dataId used above
#whereString = "tract = 9813 AND skymap = 'hsc_rings_v1'"
whereString = "exposure.day_obs={} and exposure.seq_num={} and instrument='LATISS'".format("20220607","195") 

# Prevent the executor from dumping plots into the notebook
backend_ =  mpl.get_backend() 
mpl.use("Agg")

executor = SimplePipelineExecutor.from_pipeline(pipeline, where=whereString, butler=butlerRW)
quanta = executor.run(True)

# Restore the ability for plots to be put into the notebook
mpl.use(backend_)

# If you only want to run one plot in a task in the pipeline do the following prior to execution
# pipeline.addConfigOverride("analyzeObjectTableCore", "plots", None)
# pipeline.addConfigOverride("analyzeObjectTableCore", "plots", ShapeSizeFractionalDiffScatterPlot)

Overriding default configuration file with /cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/dustmaps_cachedata/g41a3ec361e+62d12e78cb/config/.dustmapsrc


LookupError: DatasetType 'atlas_refcat2_20220201' referenced by CalibrateConnections uses 'skypix' as a dimension placeholder, but does not already exist in the registry.  Note that reference catalog names are now used as the dataset type name instead of 'ref_cat'.

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='postISRCCD', collections = outputCollection , where= "instrument='LATISS'")